In [61]:
import numpy as np
import pandas as pd 
import math as ma
from scipy import stats
from scipy.stats import beta
from scipy.optimize import fsolve

In [68]:
musun = 132712000000
T = 365.25 * 86400 * 2 / 3
e = 581.2392124070273


def f(x):
    return x**2 -8


x = fsolve(f, 0.01)

print(x)

[2.82842712]


In [171]:
# Mean_of_Matrix requires matrix as input with each sample as a row
# and output a matrix
def Mean_of_Matrix(Matrix):
    row = Matrix.shape[0]
    one = np.matrix(np.ones(row))
    mean_of_Matrix=1/row * one * Matrix
    return mean_of_Matrix

# Covariance_of_Matrix requires matrix as input and outp a matrix
def Covariance_of_Matrix(Matrix):
    row = Matrix.shape[0]
    one = np.matrix(np.ones(row))
    mean_of_Matrix = Mean_of_Matrix(Matrix)
    matrix = Matrix - mean_of_Matrix
    return matrix.transpose() * matrix / row

def Variance_of_Matrix(Matrix):
    co_matrix = Covariance_of_Matrix(Matrix)
    n=co_matrix.shape[0]
    variance = np.matrix(np.ones(n))
    for i in range(0,n):
        variance[0,i] = co_matrix[i,i]
    return variance

# T_test requires Matrix_1 & Matrix_2 have same number of columns
def T_test(Matrix_1,Matrix_2):
    N_1,N_2 = Matrix_1.shape[0], Marix_2.shape[0]
    X_A,X_B = Mean_of_Matrix(Matrix_1), Mean_of_Matrix(Matrix_2)
    S_A,S_B = Variance_of_Matrix(Matrix_1),Variance_of_Matrix(Matrix_2)
    T_trace = []
    for i in range(0,Marix_2.shape[1]):
        T_trace.append((X_A[0,i] - X_B[0,i])/ma.sqrt(S_A[0,i]/N_1 +S_B[0,i]/N_2))
    return T_trace
# Can also use 'stats.ttest_ind(matrix_1, matrix_2, equal_var = False)' from package scipy

def threshold(T_test,c):
    index = []
    t_test = T_test.pvalue.tolist()
    for i in range(0,len(t_test)):
        if abs(t_test[i])>=c :
            index.append(i)
    return index

In [173]:
# Example
matrix_1 = np.zeros((500,10))
for i in range(0,10):
    matrix_1[:,i] = stats.norm.rvs(loc=5,scale=10,size=500)
matrix_2 = np.zeros((200,10))
for i in range(0,10):
    matrix_2[:,i] = stats.norm.rvs(loc=8,scale=20,size=200)
result=stats.ttest_ind(matrix_1, matrix_2, equal_var = False)

print(threshold(result,0.1))

[3, 4, 6, 7, 8, 9]


In [51]:
# Higher criticism Algorithm
# x is array, y is number
def f_1(x,y):
    return (x+(y**2-y*(y**2+4*np.multiply(1-x, x))**0.5)/2)/(1+y**2)

def f_2(x,y):
    z = (1+y**2) * (y**2 + 4*np.multiply(1-x, x))**0.5
    return 1/(1+y**2) - np.divide(y*(1-2*x),z)

def hcpvalue(nl,th):
    k = np.array(range(0,ma.floor(nl/2)))+1
    c1 = f_1(k/nl,th/nl**0.5)
    c2 = f_2(k/nl,th/nl**0.5)
    beta_pdf = beta.pdf(c1,k,nl+1-k)
    return sum(np.divide(np.multiply(np.multiply(np.multiply(beta_pdf,np.divide(c1,k)),(1-(1-k/nl))),c2),(1-c1)))

def hctest(x,alpha=0.01):
    if alpha<=0 or alpha >=1 :
        return "BadAlpha"
    else:
        nl = len(x)
        hc_threshold = 0.5
        x0=0.1
        def fun(th):
            return hcpvalue(nl,th) - alpha
        while hc_threshold < 1.07:
            x0 = x0*10
            hc_threshold = fsolve(fun, x0)
        x_sort = sorted(x)
        hc = ma.sqrt(nl) * np.divide(((np.array(range(0,nl))+1)/nl - x_sort),(np.multiply(x_sort,(1-x_sort+1e-50)))**0.5)
        hc_max = max(hc[0:ma.floor(nl/2)])
        if hc_max > hc_threshold:
            return 1, threshold
        else:
            return 0, threshold